## Этап 1

In [1]:
import cudf.pandas  # Импортирование cuDF и активация его использования
cudf.pandas.install()  # Установка cuDF как основного интерфейса для pandas
import os
import pandas as pd
from tqdm import tqdm
import torch

In [2]:
import os
import gdown

def download_file_if_not_exists(file_url, output_path):
    """Скачивает файл с Google Drive, если он ещё не существует в указанной директории."""
    # Проверка наличия файла
    if os.path.exists(output_path):
        print(f"Файл '{output_path}' уже существует.")
    else:
        print(f"Файл '{output_path}' не найден. Начинаю загрузку...")
        gdown.download(file_url, output_path, quiet=False)
        print(f"Файл '{output_path}' успешно загружен.")

# Указываем URL и путь к файлу
# file_url = 'https://drive.google.com/uc?id=15pofNbomaoUap41Rcn1uNGeiJIqFd2qe'
file_url = 'https://drive.google.com/uc?id=1alondqI-2IHo__mYU7KQz4Ip8ytYGHXg'
output_file_name = 'wildberries_reviews.csv'  # Укажите реальное имя файла, которое хотите сохранить
output_path = os.path.join(os.getcwd(), output_file_name)  # Полный путь к файлу

download_file_if_not_exists(file_url, output_path)


Файл '/workspace/wildberries_reviews.csv' уже существует.


In [3]:
# Путь к папке с CSV файлами
folder_path = './reviews_keywords/corrected_reviews'

# Получаем список всех файлов в папке
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Читаем и объединяем все CSV файлы в один датафрейм
df_list = [pd.read_csv(os.path.join(folder_path, file), index_col="id") for file in csv_files]
combined_df = pd.concat(df_list, ignore_index=False)

combined_df.index = combined_df.index - 1
combined_df = pd.concat([pd.read_csv("wildberries_reviews.csv")[["corrected_text"]], combined_df], ignore_index=False)
# Выводим первые несколько строк объединенного датафрейма для проверки
combined_df.describe()


,corrected_text
count,362145
unique,244853
top,Все отлично
freq,87


In [4]:
combined_df

,corrected_text
0,Работает хорошо.
1,"Пришло быстро, все целое на вид. Завтра буду и..."
2,"Купил на квадр для поднятия отвала, установка ..."
3,Лебёдка хорошая. Но в инструкции ни слова про ...
4,"Всё в комплекте, есть инструкция на русском яз..."
...,...
1479799,"Пистолет супер, очень качественный. Светится, ..."
1479819,"Пистолет пришел, хорошо упакован, комплектация..."
1479834,"Классный пистолет, выпускает огромное количест..."
1479841,"Отличный пистолет, качественный👌Ребёнок в вост..."


In [5]:
df_raw_big = pd.read_csv("wildberries_reviews.csv.gz", compression="gzip").drop("Unnamed: 0", axis=1)[:1479867]
df_raw_big.head()

,review_full_text,review_rating,product,category,url
0,Работает хорошо.,5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...
1,"Пришло быстро, все целое на вид. Завтра буду и...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...
2,"Купил на квадр для поднятия отвала, установка ...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...
3,Лебёдка хорошая. Но в инструкции ни слова про ...,5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...
4,"Всё в комплекте, есть инструкция на русском яз...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...


In [6]:
result = combined_df.merge(df_raw_big, left_index=True, right_index=True, how='right')
result.describe()

,review_rating
count,1.479877e+06
mean,4.607101e+00
std,1.014978e+00
min,1.000000e+00
25%,5.000000e+00
50%,5.000000e+00
75%,5.000000e+00
max,5.000000e+00


In [7]:
result['corrected_text'] = result['corrected_text'].fillna(result['review_full_text'])

In [8]:
result.head()

,review_full_text,review_rating,product,category,url,corrected_text
0,Работает хорошо.,5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,Работает хорошо.
1,"Пришло быстро, все целое на вид. Завтра буду и...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Пришло быстро, все целое на вид. Завтра буду и..."
2,"Купил на квадр для поднятия отвала, установка ...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Купил на квадр для поднятия отвала, установка ..."
3,Лебёдка хорошая. Но в инструкции ни слова про ...,5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,Лебёдка хорошая. Но в инструкции ни слова про ...
4,"Всё в комплекте, есть инструкция на русском яз...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Всё в комплекте, есть инструкция на русском яз..."


In [9]:
# Оставляем только по 5 записей для каждого уникального значения в столбце 'product'
# result_limited = result.groupby('product').apply(lambda x: x.iloc[5:8]).reset_index(drop=True)
result_limited = result[result.category == result.iloc[-260000].category]


In [10]:
import spacy
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.cluster import DBSCAN
import numpy as np
from collections import Counter

# Проверка доступности GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Загрузка и настройка модели SpaCy
nlp = spacy.load("ru_core_news_lg")

# Пример загрузки данных в pandas DataFrame
df_raw = pd.read_csv("wildberries_reviews.csv", nrows=30000)
# df = df_raw[-500:-1]  # Отбор 500 записей для обработки
df = result_limited

# Преобразование pandas DataFrame в Hugging Face Dataset
dataset = Dataset.from_pandas(df)

import re

def clean_text(text):
    # Сначала заменяем все \n, \r, \t на пробел
    text = re.sub(r'[\n\r\t]+', ' ', text)
    
    # Удаляем лишние пробелы
    text = re.sub(r'\s{2,}', ' ', text)

    # Заменяем пробел и точку (если точка отсутствует)
    text = re.sub(r'(?<!\.)\s*\.\s*', '. ', text)  # Убедимся, что после замены есть точка и пробел
    text = re.sub(r'\s*\.\s*(?!\.)', ' ', text)  # Удаляем лишние пробелы перед точкой, если точка есть
    
    # Если текст заканчивается точкой, убираем её
    text = re.sub(r'\s*\.$', '', text)

    return text.strip()


# Функция для разбиения текста на предложения
def split_into_sentences(text):
    doc = nlp(clean_text(text))
    return [sent.text for sent in doc.sents]

# Применение функции для разбиения отзывов на предложения
def split_reviews_into_sentences(batch):
    batch['sentences'] = [split_into_sentences(text) for text in batch['corrected_text']]
    return batch

dataset = dataset.map(split_reviews_into_sentences, batched=True, batch_size=8)

# Преобразуем Dataset обратно в pandas DataFrame
df = dataset.to_pandas()

# Выполним explode по столбцу с предложениями
df_exploded = df.explode('sentences').reset_index(drop=True)

# Удаляем лишние столбцы, которые появились после explode
df_exploded = df_exploded.drop(columns=[col for col in df_exploded.columns if col.startswith('__index_level_')])

# Преобразуем DataFrame обратно в Hugging Face Dataset
dataset_exploded = Dataset.from_pandas(df_exploded)

Map:   0%|          | 0/2080 [00:00<?, ? examples/s]

In [11]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizerFast, BertForSequenceClassification
from tqdm import tqdm
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Проверяем, поддерживает ли устройство FP16
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Загрузка дообученной модели и токенизатора
tokenizer = BertTokenizerFast.from_pretrained('./reviews_keywords/fine_tuned_model')
model = BertForSequenceClassification.from_pretrained('./reviews_keywords/fine_tuned_model').to(device)

# Перевод модели в режим FP16, если это возможно
if device.type == "cuda":
    model = model.half()

# Пример данных (замените на реальные данные)
reviews = dataset_exploded["sentences"]

# Создание кастомного Dataset для обработки отзывов
class ReviewDataset(Dataset):
    def __init__(self, reviews, tokenizer, max_len=128):
        self.reviews = reviews
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.reviews)

    def __getitem__(self, idx):
        review = self.reviews[idx]
        encoding = self.tokenizer.encode_plus(
            review,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        return {key: val.flatten() for key, val in encoding.items()}

# Создаем датасет и DataLoader
dataset = ReviewDataset(reviews, tokenizer)
batch_size = 128
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

# Получение предсказаний с отображением прогресса
predictions = []

for batch in tqdm(dataloader, desc="Предсказание отзывов"):
    batch = {key: val.to(device) for key, val in batch.items()}

    with torch.no_grad():
        outputs = model(**batch)
        logits = outputs[0] if isinstance(outputs, tuple) else outputs.logits
        probabilities = torch.softmax(logits, dim=-1)
        batch_predictions = (probabilities[:, 1] > 0.9).cpu().numpy()  # Используем порог 0.6 вместо 0.5
        predictions.extend(batch_predictions)

# Преобразование в DataFrame, если это еще не сделано
if not isinstance(dataset_exploded, pd.DataFrame):
    dataset_exploded = pd.DataFrame(dataset_exploded)

# Присоединение предсказаний к датасету
dataset_exploded['predictions'] = predictions
dataset_exploded.head()


Предсказание отзывов: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [00:13<00:00,  1.54it/s]


,review_full_text,review_rating,product,category,url,corrected_text,sentences,__index_level_0__,predictions
0,"Чумовая кепка, с мужем поделить не можем, прид...",5,Limastar accessories / Кепка летняя New York N...,/Женщинам/Пляжная мода/Аксессуары,https://www.wildberries.ru/catalog/208661951/f...,"Чумовая кепка, с мужем поделить не можем, прид...","Чумовая кепка, с мужем поделить не можем, прид...",0,True
1,"Чумовая кепка, с мужем поделить не можем, прид...",5,Limastar accessories / Кепка летняя New York N...,/Женщинам/Пляжная мода/Аксессуары,https://www.wildberries.ru/catalog/208661951/f...,"Чумовая кепка, с мужем поделить не можем, прид...",заказать,1,True
2,"Ну хорошая кепка, но только козырек немного дл...",5,Limastar accessories / Кепка летняя New York N...,/Женщинам/Пляжная мода/Аксессуары,https://www.wildberries.ru/catalog/208661951/f...,"Ну хорошая кепка, но только козырек немного дл...","Ну хорошая кепка, но только козырек немного дл...",2,True
3,"Нууу такая себе, посадка не глубокая, затянула...",4,Limastar accessories / Кепка летняя New York N...,/Женщинам/Пляжная мода/Аксессуары,https://www.wildberries.ru/catalog/208661951/f...,"Нууу такая себе, посадка не глубокая, затянула...","Нууу такая себе, посадка не глубокая, затянула...",3,True
4,"Кепка бесформенная, передний шов косой, на гол...",3,Limastar accessories / Кепка летняя New York N...,/Женщинам/Пляжная мода/Аксессуары,https://www.wildberries.ru/catalog/208661951/f...,"Кепка бесформенная, передний шов косой, на гол...","Кепка бесформенная, передний шов косой, на гол...",4,True


In [12]:
dataset_exploded[dataset_exploded.predictions == 1]

,review_full_text,review_rating,product,category,url,corrected_text,sentences,__index_level_0__,predictions
0,"Чумовая кепка, с мужем поделить не можем, прид...",5,Limastar accessories / Кепка летняя New York N...,/Женщинам/Пляжная мода/Аксессуары,https://www.wildberries.ru/catalog/208661951/f...,"Чумовая кепка, с мужем поделить не можем, прид...","Чумовая кепка, с мужем поделить не можем, прид...",0,True
1,"Чумовая кепка, с мужем поделить не можем, прид...",5,Limastar accessories / Кепка летняя New York N...,/Женщинам/Пляжная мода/Аксессуары,https://www.wildberries.ru/catalog/208661951/f...,"Чумовая кепка, с мужем поделить не можем, прид...",заказать,1,True
2,"Ну хорошая кепка, но только козырек немного дл...",5,Limastar accessories / Кепка летняя New York N...,/Женщинам/Пляжная мода/Аксессуары,https://www.wildberries.ru/catalog/208661951/f...,"Ну хорошая кепка, но только козырек немного дл...","Ну хорошая кепка, но только козырек немного дл...",2,True
3,"Нууу такая себе, посадка не глубокая, затянула...",4,Limastar accessories / Кепка летняя New York N...,/Женщинам/Пляжная мода/Аксессуары,https://www.wildberries.ru/catalog/208661951/f...,"Нууу такая себе, посадка не глубокая, затянула...","Нууу такая себе, посадка не глубокая, затянула...",3,True
4,"Кепка бесформенная, передний шов косой, на гол...",3,Limastar accessories / Кепка летняя New York N...,/Женщинам/Пляжная мода/Аксессуары,https://www.wildberries.ru/catalog/208661951/f...,"Кепка бесформенная, передний шов косой, на гол...","Кепка бесформенная, передний шов косой, на гол...",4,True
...,...,...,...,...,...,...,...,...,...
2618,"Как мама сказала:"" О, это русская сумка ""😁\nПо...",5,S.LAVIA / Сумка через плечо маленькая кросс-боди,/Женщинам/Пляжная мода/Аксессуары,https://www.wildberries.ru/catalog/12487419/fe...,"Как мама сказала:"" О, это русская сумка ""😁\nПо...","Как мама сказала:"" О, это русская сумка ""😁 Пок...",2618,True
2622,"Пришла в идеальном состоянии, очень вместитель...",5,S.LAVIA / Сумка через плечо маленькая кросс-боди,/Женщинам/Пляжная мода/Аксессуары,https://www.wildberries.ru/catalog/12487419/fe...,"Пришла в идеальном состоянии, очень вместитель...","Пришла в идеальном состоянии, очень вместитель...",2622,True
2624,"Сумка понравилась. Удобная, вместительная) за ...",5,S.LAVIA / Сумка через плечо маленькая кросс-боди,/Женщинам/Пляжная мода/Аксессуары,https://www.wildberries.ru/catalog/12487419/fe...,"Сумка понравилась. Удобная, вместительная) за ...","надеюсь, быстро не облезет",2624,True
2625,"Маленькая, но вместительная. Аккуратно сшита, ...",5,S.LAVIA / Сумка через плечо маленькая кросс-боди,/Женщинам/Пляжная мода/Аксессуары,https://www.wildberries.ru/catalog/12487419/fe...,"Маленькая, но вместительная. Аккуратно сшита, ...","Маленькая, но вместительная Аккуратно сшита, в...",2625,True


In [13]:
import os
import numpy as np
import pandas as pd
from sklearn.cluster import DBSCAN
from sklearn.metrics.pairwise import cosine_similarity
import torch
from transformers import BertTokenizerFast, BertForSequenceClassification, BertConfig
import nltk
from nltk.corpus import stopwords
import spacy
from tqdm import tqdm
import logging

# Отключение параллелизма в токенайзере Hugging Face
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Устройство (GPU или CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Настройка логирования
logging.basicConfig(filename='./reviews_keywords/clustering.log', 
                    level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s')

# Загрузка модели spaCy для русского языка
nlp = spacy.load("ru_core_news_lg")

# Установка стоп-слов
nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))

# Настройка конфигурации модели для возврата скрытых состояний
config = BertConfig.from_pretrained('./reviews_keywords/fine_tuned_model', output_hidden_states=True)
tokenizer = BertTokenizerFast.from_pretrained('./reviews_keywords/fine_tuned_model')
model = BertForSequenceClassification.from_pretrained('./reviews_keywords/fine_tuned_model', config=config).to(device)

# Функция для вычисления центра кластера (центроида)
def find_centroid(embeddings):
    return np.mean(embeddings, axis=0)

# Функция для нахождения ключевой мысли в кластере
def extract_key_thought(cluster_sentences):
    sentences = cluster_sentences.split(" | ")
    
    embeddings = compute_sentence_embeddings(sentences)
    
    centroid = find_centroid(embeddings)
    similarities = cosine_similarity(embeddings, [centroid])
    key_sentence_index = np.argmax(similarities)
    
    return sentences[key_sentence_index]

# Функция для подсчета количества слов в каждом кластере
def count_words(cluster_sentences):
    words = cluster_sentences.split()
    return len(words)

# Функция для вычисления эмбеддингов
def compute_sentence_embeddings(sentences):
    inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        # Получаем скрытые состояния
        hidden_states = outputs.hidden_states[-1]
    embeddings = hidden_states.mean(dim=1).cpu().numpy()
    return embeddings

# Функция для повторной кластеризации крупных кластеров
def recluster_large_cluster(cluster_sentences, eps=0.1, min_samples=2):
    sentences = cluster_sentences.split(" | ")
    
    embeddings = compute_sentence_embeddings(sentences)
    
    re_clustering = DBSCAN(eps=eps, min_samples=min_samples, metric="cosine").fit(embeddings)
    
    re_cluster_dict = {}
    for idx, label in enumerate(re_clustering.labels_):
        if label == -1:
            continue
        label_str = str(label)
        if label_str not in re_cluster_dict:
            re_cluster_dict[label_str] = []
        re_cluster_dict[label_str].append(sentences[idx])
    
    return [" | ".join(cluster) for cluster in re_cluster_dict.values()]

# Рекурсивная функция для кластеризации крупных кластеров
def recursive_clustering(cluster_sentences, threshold, eps=0.25, min_samples=3, min_eps=0.05):
    current_eps = eps
    new_clusters = [cluster_sentences]

    while True:
        next_clusters = []
        reclustered_any = False
        
        for cluster in new_clusters:
            if count_words(cluster) > threshold:
                while current_eps >= min_eps:
                    reclustered = recluster_large_cluster(cluster, eps=current_eps, min_samples=min_samples)
                    if len(reclustered) > 1:
                        next_clusters.extend(reclustered)
                        reclustered_any = True
                        break  # Кластер успешно разделен, выходим из внутреннего цикла
                    else:
                        current_eps -= 0.02  # Уменьшаем eps и пробуем снова
                
                if len(reclustered) == 1:
                    # Если кластер так и не был разделен, добавляем его обратно
                    next_clusters.append(cluster)
            else:
                next_clusters.append(cluster)
        
        new_clusters = next_clusters
        
        if not reclustered_any:
            break
    
    return new_clusters

# Основной процесс кластеризации по товарам
final_result = pd.DataFrame()

for product_name, group in dataset_exploded[dataset_exploded["predictions"] == 1].groupby('product'):
    all_sentences = group['sentences'].tolist()

    # Обработка предложений без разделения на батчи
    all_embeddings = compute_sentence_embeddings(all_sentences)

    # Прогресс-бар для начальной кластеризации
    clustering = DBSCAN(eps=0.25, min_samples=3, metric="cosine").fit(all_embeddings)

    cluster_dict = {}
    for idx, label in tqdm(enumerate(clustering.labels_), desc=f"Organizing clusters for {product_name}"):
        if label == -1:
            continue
        label_str = str(label)
        if label_str not in cluster_dict:
            cluster_dict[label_str] = set()
        cluster_dict[label_str].add(all_sentences[idx])

    clusters = [" | ".join(sentences) for sentences in cluster_dict.values()]
    threshold = np.min([np.mean([count_words(cluster) for cluster in clusters]) * 1.5  ,  450])

    final_clusters = []
    for cluster in tqdm(clusters, desc="Recursive clustering"):
        final_clusters.extend(recursive_clustering(cluster, threshold))

    df_exploded_sorted = pd.DataFrame({'product': product_name, 'cluster_sentences': final_clusters})
    df_exploded_sorted['word_count'] = df_exploded_sorted['cluster_sentences'].apply(count_words)
    df_exploded_sorted['key_thought'] = df_exploded_sorted['cluster_sentences'].apply(extract_key_thought)

    df_exploded_sorted = df_exploded_sorted.sort_values(by='word_count', ascending=False)

    final_result = pd.concat([final_result, df_exploded_sorted], ignore_index=True)

# Показать результат
display(final_result[['product', 'cluster_sentences', 'key_thought', 'word_count']])


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Organizing clusters for Averkator / Кепка летняя спортивная: 116it [00:00, 712356.17it/s]
Recursive clustering: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.83s/it]
Organizing clusters for Binoni / Кепка женская бейсболка летняя козырек от солнца для спорта: 111it [00:00, 683653.07it/s]
Recursive clustering: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:07<00:00,  7.67s/it]
Organizing clusters for Femberry / Бейсболка с принтом: 31it [00:00, 220005.79it/s]
Recur

,product,cluster_sentences,key_thought,word_count
0,Averkator / Кепка летняя спортивная,"Отличная кепка, хорошего качества Цвет красивы...","Хорошая,качественная кепка Нитки не где не тор...",50
1,Averkator / Кепка летняя спортивная,Классная кепка! | Отличная кепка ! | Отличная ...,Отличная кепка !,12
2,Binoni / Кепка женская бейсболка летняя козыре...,Кепка классная 👍🏻 | Замечательная кепка Совету...,"Отличная кепка, смотрится здорово!",83
3,Binoni / Кепка женская бейсболка летняя козыре...,"Интересная кепка, чуть великовата, не критично...","Красиво смотрится и на короткие волосы, козыре...",36
4,Binoni / Кепка женская бейсболка летняя козыре...,"Козырёк отличный! | козырёк | Хороший козырек,...",Классный козырек!👍,15
5,Femberry / Бейсболка с принтом,"Красивая кепка , но огровная не знаю на какую ...","Кепка неплохая, но на 56 размер великовата, гл...",245
6,Fishka / Кепка женская белая,"Хорошая кепка, упакована хорошо - пришла в пак...","Хорошая кепка, упакована хорошо - пришла в пак...",111
7,Fishka / Кепка женская белая,Отличная кепка! | Кепка хорошая | Отличная кеп...,Очень крутая кепка ✅👍,53
8,G.T / Бейсболка без козырька кепка докер варен...,"абалденная, советую, не пожалеете и по размеру...","Не чисто белая, скорее молочная Нам с мужем по...",264
9,G.T / Бейсболка без козырька кепка докер варен...,Довольно качественная работа Хорошая ткань Пос...,"Кепка отличного качества, хб, и нет твёрдой вс...",154
